In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from functools import partial

In [ ]:
# use this to keep the cells wider, very nice on wide screens, set the percentage as you like it
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))
#display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:

conf = {
    'font.size': 14.0,
    'axes.grid': True,
    'axes.axisbelow': True,
    'axes.edgecolor': 'black',
#    'axes.facecolor': '#E5E5EF',
#    'axes.facecolor': '#E0E0EF',
    'axes.labelcolor': 'black',
    'axes.titlesize': 15.6,
    'axes.labelsize': 'large',    
    'figure.figsize': (12, 7),
    'figure.titlesize': 'x-large',
    'grid.linewidth': 1.3,
    'xtick.labelsize': 'large',
    'ytick.labelsize': 'large',
    'xtick.color': 'black',
    'ytick.color': 'black',
    'legend.fontsize': 'large',
    'figure.figsize': (12., 8.)
}

plt.rcParams.update(conf)

### base loading and cleaning

In [ ]:
df_form = pd.read_excel('Feedback_Animefest_2019.xlsx', sheet_name='Form Responses 1')

In [ ]:
df_form['Timestamp'] = pd.to_datetime(df_form['Timestamp'])

In [ ]:
df_app = pd.read_excel('Feedback_Animefest_2019.xlsx', sheet_name='Appky')

In [ ]:
df_app['čas feedbacku'] = pd.to_datetime(df_app['čas feedbacku'])

In [ ]:
df_form = df_form.replace(to_replace={'😐v pohodě': 'v pohodě', '😃dobré': 'dobré', '😞špatné': 'špatné', '😐 v pohodě': 'v pohodě', '😃 dobré': 'dobré', '😞 špatné': 'špatné', '😃Ano': 'Ano', '😞Ne': 'Ne'})

In [ ]:
df_form.head()

##### several columns with same name, mutually exlusively filled, merging those

In [ ]:
duplicit_columns = [i for i in df_form.columns if i.endswith('.1')]
duplicit_columns

In [ ]:
[df_form[[i[:-2], i]].notnull().sum(axis=1).max() for i in duplicit_columns]

In [ ]:
for i in duplicit_columns:
    assert df_form[[i[:-2], i]].notnull().sum(axis=1).max() == 1  # only one column is filled, the other is N/A
    df_form.loc[df_form[i].notnull(), i[:-2]] = df_form[df_form[i].notnull()][i]
df_form = df_form.drop(columns=duplicit_columns)

In [ ]:
info_suffix = ' [Dozvěděl(a) jsem se zajímavé informace]'
fun_suffix = ' [Bavil(a) jsem se]'
columns_series = df_form.columns.to_series()
program_columns_ratings = columns_series[columns_series.str.contains(info_suffix, regex=False) | columns_series.str.contains(fun_suffix, regex=False)]
program_columns_ratings.head()

In [ ]:
program_columns = pd.Series(program_columns_ratings.str.replace(info_suffix, '', regex=False).str.replace(fun_suffix, '', regex=False).unique())
program_columns.head()

##### some columns that should have same content have nans somewhere, fixing this

In [ ]:
weird_cols = ['2. světová válka z pohledu Japonska a život císaře Hirohita [Dozvěděl(a) jsem se zajímavé informace]', '2. světová válka z pohledu Japonska a život císaře Hirohita [Bavil(a) jsem se]']
df_form[df_form[weird_cols[0]] == 'Nedostal(a) jsem se'][weird_cols].head()

In [ ]:
# if one column marks not attended, and second is null, value is inferred
for i in program_columns:
    info_cond = df_form[i+info_suffix].isna() & df_form[i+fun_suffix].isin(['Nedostal(a) jsem se', 'Nezúčastnil(a) jsem se'])
    fun_cond = df_form[i+fun_suffix].isna() & df_form[i+info_suffix].isin(['Nedostal(a) jsem se', 'Nezúčastnil(a) jsem se'])
    df_form.loc[info_cond, i+info_suffix] = df_form[info_cond][i+fun_suffix]
    df_form.loc[fun_cond, i+fun_suffix] = df_form[fun_cond][i+info_suffix]


In [ ]:
df_form[df_form[weird_cols[0]] == 'Nedostal(a) jsem se'][weird_cols].head()

##### some columns that should have nan or rating have not attended, fixing this

In [ ]:
weird_cols = ['Maul Cosplay Q&A [Dozvěděl(a) jsem se zajímavé informace]', 'Maul Cosplay Q&A [Bavil(a) jsem se]']
df_form[df_form[weird_cols[1]] == 'v pohodě'][weird_cols].head()

In [ ]:
# if one column marks not attended, and second is rating, not attended is set to null
for i in program_columns:
    info_cond = df_form[i+fun_suffix].isin(['dobré', 'v pohodě', 'špatné']) & df_form[i+info_suffix].isin(['Nedostal(a) jsem se', 'Nezúčastnil(a) jsem se'])
    fun_cond = df_form[i+info_suffix].isin(['dobré', 'v pohodě', 'špatné']) & df_form[i+fun_suffix].isin(['Nedostal(a) jsem se', 'Nezúčastnil(a) jsem se'])
    df_form.loc[info_cond, i+info_suffix] = np.nan
    df_form.loc[fun_cond, i+fun_suffix] = np.nan


In [ ]:
df_form[df_form[weird_cols[1]] == 'v pohodě'][weird_cols].head()

##### some columns that should have same type of attendence have different. Converting 'Nedostal(a) jsem se' to 'Nezúčastnil(a) jsem se'

In [ ]:
weird_cols = ['12 ran cosplayerových [Dozvěděl(a) jsem se zajímavé informace]', '12 ran cosplayerových [Bavil(a) jsem se]']
df_form[df_form[weird_cols[0]] == 'Nedostal(a) jsem se'][weird_cols].tail()

In [ ]:
# if one column marks not attended, and second is null, value is inferred
for i in program_columns:
    info_cond = (df_form[i+info_suffix] == 'Nedostal(a) jsem se') & (df_form[i+fun_suffix] == 'Nezúčastnil(a) jsem se')
    fun_cond = (df_form[i+fun_suffix] == 'Nedostal(a) jsem se') & (df_form[i+info_suffix] == 'Nezúčastnil(a) jsem se')
    df_form.loc[info_cond, i+info_suffix] = df_form[info_cond][i+fun_suffix]
    df_form.loc[fun_cond, i+fun_suffix] = df_form[fun_cond][i+info_suffix]


In [ ]:
df_form[df_form[weird_cols[0]] == 'Nedostal(a) jsem se'][weird_cols].tail()

##### sanity checks

In [ ]:
for i in program_columns:
    info_sum = (df_form[i+info_suffix] == 'Nedostal(a) jsem se').sum()
    fun_sum = (df_form[i+fun_suffix] == 'Nedostal(a) jsem se').sum()
    both_sum = ((df_form[i+info_suffix] == 'Nedostal(a) jsem se') & (df_form[i+fun_suffix] == 'Nedostal(a) jsem se')).sum()
    if info_sum != fun_sum or fun_sum != both_sum:
        print(i, info_sum, fun_sum, both_sum)
    
    assert info_sum == fun_sum == both_sum

In [ ]:
for i in program_columns:
    info_sum = (df_form[i+info_suffix] == 'Nezúčastnil(a) jsem se').sum()
    fun_sum = (df_form[i+fun_suffix] == 'Nezúčastnil(a) jsem se').sum()
    both_sum = ((df_form[i+info_suffix] == 'Nezúčastnil(a) jsem se') & (df_form[i+fun_suffix] == 'Nezúčastnil(a) jsem se')).sum()
    if info_sum != fun_sum or fun_sum != both_sum:
        print(i, info_sum, fun_sum, both_sum)
    
    assert info_sum == fun_sum == both_sum

In [ ]:
for i in program_columns:
    info_sum = df_form[i+info_suffix].isin(['dobré', 'v pohodě', 'špatné', np.nan]).sum()
    fun_sum = df_form[i+fun_suffix].isin(['dobré', 'v pohodě', 'špatné', np.nan]).sum()
    both_sum = df_form[i+info_suffix].isin(['dobré', 'v pohodě', 'špatné', np.nan]).sum() & df_form[i+fun_suffix].isin(['dobré', 'v pohodě', 'špatné', np.nan]).sum()
    if info_sum != fun_sum or fun_sum != both_sum:
        print(i, info_sum, fun_sum, both_sum)
    
    assert info_sum == fun_sum == both_sum

### base EDA

In [ ]:
df_form.head()

In [ ]:
df_form.describe()

In [ ]:
[(i, len(df_form[i].unique())) for i in df_form.columns if len(df_form[i].unique()) < 10]

In [ ]:
df_form.columns.map(lambda i: len(df_form[i].unique())).to_series().value_counts().sort_index().plot.bar(figsize=(25, 10))

In [ ]:
df_form.columns.map(lambda i: sum(df_form[i].notnull())).to_series().value_counts().sort_index().plot.bar(figsize=(25, 10))

In [ ]:
attend_columns = [i for i in df_form.columns if 'Nezúčastnil(a) jsem se' in df_form[i].values]

In [ ]:
attend_columns[:10]

In [ ]:
len(attend_columns)

In [ ]:
len([i for i in df_form.columns if len(df_form[i].unique()) <= 8])

In [ ]:
len([i for i in df_form.columns if len(df_form[i].unique()) < 10])

In [ ]:
len([i for i in df_form.columns if len(df_form[i].unique()) < 20])

In [ ]:
len([i for i in df_form.columns if len(df_form[i].unique()) <= 8 and sum(df_form[i].notnull()) > 100])

In [ ]:
all_attends = df_form[attend_columns].stack()
all_attends[~all_attends.isna()].value_counts().plot.pie()

In [ ]:
all_attends[~all_attends.isna()].value_counts().plot.barh()

In [ ]:
all_attends[(~all_attends.isna()) & ('Nezúčastnil(a) jsem se' != all_attends)].value_counts().plot.barh()

In [ ]:
df_app.head()

In [ ]:
df_app.describe()

In [ ]:
def abs_vals(pct, allvals):
    absolute = int(pct/100.*np.sum(allvals))
    return '{:d}'.format(absolute)


In [ ]:
feedback_geo = df_form['Bydliště'].value_counts()
feedback_geo.plot.pie(autopct=partial(abs_vals, allvals=feedback_geo))

In [ ]:
for i in [i for i in df_form.columns if len(df_form[i].unique()) <= 8 and sum(df_form[i].notnull()) > 100]:
    plt.figure()
    feedback_i = df_form[i].value_counts()
    feedback_i.plot.pie(autopct=partial(abs_vals, allvals=feedback_i))
    plt.show()
    
# todo: dodat filtrování, že tam kde je pod 20 co se zúčastnilia ty vyhodit, udělat i analýzu toho, kdo jak vyplnil zúčastnil/nezúčastnil jsem se

In [ ]:
[i for i in df_form.columns if len(df_form[i].unique()) <= 8 and '[Dozvěděl(a) jsem se zajímavé informace]' not in i and '[Bavil(a) jsem se]' not in i and 'Komentář:' not in i]

In [ ]:
cols_to_plot = [i for i in df_form.columns if len(df_form[i].unique()) <= 8 and '[Dozvěděl(a) jsem se zajímavé informace]' not in i and '[Bavil(a) jsem se]' not in i and 'Komentář:' not in i]
cols = 3
fig, axes = plt.subplots(int(np.ceil(len(cols_to_plot) / cols)), cols, figsize=(35, 120))

for i, col in enumerate(cols_to_plot):
    ax = axes[i // cols, i % cols]
    #plt.figure()
    feedback_i = df_form[col].value_counts()
    feedback_i.plot.pie(autopct=partial(abs_vals, allvals=feedback_i), ax=ax, y=None)
plt.show()
    
# todo: dodat filtrování, že tam kde je pod 20 co se zúčastnilia ty vyhodit, udělat i analýzu toho, kdo jak vyplnil zúčastnil/nezúčastnil jsem se

In [ ]:
feedback_times = df_form['Timestamp'].value_counts(sort=False).copy().reindex().resample('1D', how='count')
feedback_times.plot.bar()
plt.title('Počet vyplnění za den, vše červen')
#plt.gca().set_xticklabels([dt.strftime("%#d. %#m.") for dt in feedback_times.index])
plt.gca().set_xticklabels([dt.strftime("%#d.") for dt in feedback_times.index])
plt.xticks(rotation=0)
plt.show()

In [ ]:
program_rating_columns = [i for i in df_form.columns if '[Dozvěděl(a) jsem se zajímavé informace]' in i or '[Bavil(a) jsem se]' in i]

In [ ]:
program_ratings = df_form[program_rating_columns].apply(lambda x: x.value_counts(), axis=0)

In [ ]:
attended_col = (program_ratings.loc['dobré'].fillna(0) + program_ratings.loc['v pohodě'].fillna(0) + program_ratings.loc['špatné'].fillna(0)).rename('Zúčastnil(a) jsem se')
program_attends = pd.concat((attended_col, program_ratings.loc['Nedostal(a) jsem se']), axis=1)
program_attends.head()

In [ ]:
program_ratings = program_ratings.drop('Nezúčastnil(a) jsem se')

In [ ]:
df_form[program_rating_columns].shape, len(program_rating_columns)

In [ ]:
program_ratings

In [ ]:
def remove_suffix(string):
    return string.replace(' [Dozvěděl(a) jsem se zajímavé informace]', '').replace(' [Bavil(a) jsem se]', '')
program_ratings.groupby(program_ratings.columns.map(remove_suffix), axis=1).max()

In [ ]:
program_attends.T.groupby(program_ratings.columns.map(remove_suffix), axis=1).max().T.sort_values('Zúčastnil(a) jsem se', ascending=True).plot.barh(stacked=True, figsize=(30, 50))

In [ ]:
program_attendance = program_attends.T.groupby(program_ratings.columns.map(remove_suffix), axis=1).max().T['Zúčastnil(a) jsem se']
program_attendance.sort_values(ascending=True).plot.barh(stacked=True, figsize=(30, 50))

In [ ]:
#### splitting program to categories

In [ ]:
categories = {
    'soutěže': [ 
        'Animekvíz', 'Cosplay debut', 'Cosplay soutěž', 'Cosplay video', 'Festdance', 'Soutěžní AMV', 'Vyhlášení výsledků', 
    ], 
    'přednášky': [
        '12 ran cosplayerových', '2. světová válka z pohledu Japonska a život císaře Hirohita', '3D tisk v cosplayi: Od modelování po barvení', 'Alternativní móda: Od ponožky po klobouk', 'An Introduction to Leather Crafting', 
        'Anime světy, ve kterých (ne)chcete žít', 'BJD od Ludvíka XIV. až po BTS', 'Bojuj podle svého charakteru', 'Cosplay and Otaku History in Japan', 'Cosplay armor: From reference to wearable costume', 
        'Crossdressing - the hell is dat?', 'Designing Male Characters Fanservice: The Struggle', 'Do Japonska za idoly', 'Doll Photography', 'Fake Is Sad/Bootleg Panel', 'Filmoví skladatelé Japonska č. 2', 
        'Gača hry aneb Lootboxy na asijský způsob', 'Gejša: Tajemství za stěnami čajovny', 'Ghibli známé neznámé', 'Godzilla, just a rubber monster, right?', 'Gothic – hudba, móda, subkultura', 
        'Historická přesnost zbrojí v anime na vybraných příkladech', 'Indonésie – země ohně', 'Jak na anime make-up', 'Jak na psaní – Od teorie k praxi', 'Jak se připravit na cosplay focení', 'Jak sehnat (nejen) jaoi v totalitním státu', 
        'Jak si plnit sny pomocí anime', 'Japanacorps, WTF?!', 'Japonský rok v emodži', 'Letem jiným světem', 'Lolitou každý den', 'Líčení pro trdla od trdla', 'Madoka Magica a ti druzí, co selhali', 'Manga novinky u nás!', 
        'Manga pro dospívající dívky: Co na ní milujeme a nesnášíme', 'Maul Cosplay Q&A', 'My Hero Academy', 'Módní přehlídka 101', 'Nebojte se zahraničních soutěží!', 'Nový kreslíř na scéně: umělá inteligence!', 
        'Očekávání vs. realita: Člověk 2.0, mecha a vesmírné lodě', 'Pen & Paper role-playing hry', 'Proč (ne)jet do Jižní Koreje', 'Receptář tetičky Mitsu nejen pro začínající lolity', 'Remaky anime', 'Reputace & renesance D&D', 
        'Resident Evil – Od zrození po současnost', 'Rámen po česku', 'Sekiro, Nioh a ti další…', 'Slasti a strasti japonského randění', 'Slavnostní zahájení + křest Vějíře', 'Stereotyp – NÁMITKA!', 'Svět japonských idolů', 
        'Taková normální japonská rodinka', 'Viktoriánské úpravy vlasů', 'World cosplay panel', 'Zakončení conu', 'Zákeřné vody videoherního pirátství', 
    ], 
    'workshopy': [
        'Aikido Ikigai Dojo Brno ', 'Aikikai Aikido Brno ', 'Boj dýkou a nožem', 'Boj tesákem', 'Choker', 'Cosplay Act Workshop', 'Yoshinkan aikido', 'Kaligrafie', 'Kendo', 'Kensei Dojo Brno ', 'Kimono workshop', 'Sraz AMV', 
        'Háčkování plyšových zvířátek', 'I pád je posun vpřed', 'Jak se při cosplayi neztrapnit se zbraní', 'Jo, bokken, tanto aneb Obrana proti japonským zbraním', 'Květiny Ikebana', 'Mák a jeho klacek aneb I monk si s holí leccos dovolí', 
        'Náušnice ze skleněných kapek', 'Od základů po pokročilejší retuše nejen cosplay fotografie', 'Omalovánky', 'Origami Workshop', 'Rozcvička pro družinku aneb Dračí doupě po ránu', 'Rytíř, meč a štít aneb Líný šerm', 
        'Růže ze saténových stužek', 'Sebeobrana nejen pro dámy', 'Sebeobrana po japonsku', 'Taneční k-pop workshop', 'Vyšívání', 'Válečník mazlivě obouručákem tě mlaskne', 'Výroba vlastního diáře/skicáku', 
        'Workshop animace v Clip Studio Paint', 'Základy svícení s profesionálními záblesky a jejich modifikátory'
    ], 
    'promítání': [
        '3D holka', 'A to je špatně, když se chodím seznamovat do kobek? Orionův šíp', 'Bůh žehnej tomuto nádhernému světu', 'Fashion Mix', 'Gamers!', 'Já malý čarodějka', 'Květina zaslíbená rannímu loučení', 'Lolita Music Mix: Novinky', 
        'Mirai, dívka z budoucnosti', 'Módní kolekce 2019', 'Noc je krátká, tak kráčej dál, děvče', 'Penguin Highway', 'Pohodářky', 'ReLIFE', 'Symfonie z jiného světa', 'Tipy pro život mimo realitu', 'Vzestup hrdiny štítu', 
        'Zombie Land Saga', 
    ],
    'doplňkový program': [
        'AMV Budíček', 'AMV Mortal Combat', 'AMV Nočník', 'AMV Večerníček', 'BTS Army sraz', 'Vějíř – autogramiáda a beseda', 'Anime novinky očima protřelých fanoušků', 'Budoucnost českých anime conů', 
        'K-pop – random dance CZHW', 'Sčítání žije!', 
    ],
    'divadlo': ['Festovní koncert', 'Kamui – The Samurai Sword Artists', 'Lolita Fashion Show', 'Malé divadlo kjógenu', ],
    'wtf': ['Turnaj v šermu Aréna Alerie', 'Dračí doupě', ]
    }

In [ ]:
print(program_attendance.index.to_list())

In [ ]:
program_ratings.T.columns

### analysis of open answers

In [ ]:
from urllib import request

data = request.urlopen('https://raw.githubusercontent.com/stopwords-iso/stopwords-cs/master/stopwords-cs.txt')
all_stopwords = [line.decode('utf-8').replace('\n', '') for line in data]
# todo: split them correctly and for vocalulary calculation use only emotes filtering
emotes_stopwords = [':d', ':', '\?', ';', ':-d', '\^\^', ':dd', 'xd', ':3', '=d', ':p',  ':/', ':>', '\+', 'o_o', 'd:', '>', ':\'d', ':"d', '\n', ':dd', ':ddd', ':dddd', 'x', 'xp']
my_stopwords = ['no', '', 'jo', 'sa', 'the', '=d', 'fakt', 'celkem', 'sem', 'ja', 'som', 'ako', 'of', 'nekdo', 'bych', 'sou', '2', 'jj', 'is', 'mam', '3', '\n']
all_stopwords += my_stopwords
all_stopwords += emotes_stopwords

def messages_to_words_hist(mess_series):
    mess_series = mess_series.fillna('')
    all_words = mess_series.str.split(' ', expand=True).stack().reset_index(drop=True, level=1)
    words_historgram = all_words.str.lower().str.strip('.,?-–()').value_counts()
    stopwords = list(set(all_stopwords).intersection(set(words_historgram.index)))
    words_historgram_cleaned = words_historgram.drop(stopwords)
    return words_historgram_cleaned


In [ ]:
best_mess = messages_to_words_hist(df_form['S čím jste byli letos nejvíce spokojeni / co se vám nejvíce líbilo?'])
best_mess[:30].plot.bar(figsize=(30, 10), title='S čím jste byli letos nejvíce spokojeni / co se vám nejvíce líbilo?')
plt.xticks(rotation=30)

In [ ]:
with pd.option_context('display.max_colwidth', 150):
    display(df_form[df_form['S čím jste byli letos nejvíce spokojeni / co se vám nejvíce líbilo?'].str.contains('přednášky', regex=False)]['S čím jste byli letos nejvíce spokojeni / co se vám nejvíce líbilo?'])

In [ ]:
best_mess = messages_to_words_hist(df_form['Co vás letos nejvíc zklamalo / nelíbilo se vám?'])
best_mess[:30].plot.bar(figsize=(30, 10), title='Co vás letos nejvíc zklamalo / nelíbilo se vám?')
plt.xticks(rotation=30)

In [ ]:
with pd.option_context('display.max_colwidth', 150):
    display(df_form[df_form['Co vás letos nejvíc zklamalo / nelíbilo se vám?'].str.contains('přednášky', regex=False)]['Co vás letos nejvíc zklamalo / nelíbilo se vám?'])

In [ ]:
best_mess = messages_to_words_hist(df_form['Co bychom podle vás měli do příštího roku zlepšit?'])
best_mess[:30].plot.bar(figsize=(30, 10), title='Co bychom podle vás měli do příštího roku zlepšit?')
plt.xticks(rotation=30)

In [ ]:
best_mess = messages_to_words_hist(df_form['Co bychom podle vás měli do příštího roku zlepšit?'])
best_mess[:7].plot.bar(figsize=(30, 10), title='Co bychom podle vás měli do příštího roku zlepšit?')
plt.xticks(rotation=30)